# Step 0 : install dependencies

In [1]:
# !pip install tensorflow==2.3.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2

# Step 1 :Test random enviroment with OpenAI Gym

In [2]:
import gym
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
actions

2

In [6]:
episodes = 10
for episode in range(1,  episodes+1):
  state =env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = random.choice([0,1])
    n_state, reward, done, info=env.step(action)
    score+=reward
  print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:12.0
Episode:2 Score:19.0
Episode:3 Score:10.0
Episode:4 Score:15.0
Episode:5 Score:67.0
Episode:6 Score:41.0
Episode:7 Score:27.0
Episode:8 Score:17.0
Episode:9 Score:11.0
Episode:10 Score:11.0


# Step 2 : Create Deep Learning model with Keras

In [21]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
def build_models(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions,activation ='linear'))
    return model

In [9]:
model = build_models(states, actions)

In [10]:
 model.summary() # visualize how our model look like

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Step 3 : Build agent with Keras-RL

---



In [22]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  #so this tutorial example use policy based model
from rl.memory import SequentialMemory

In [23]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [24]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 11:25 - reward: 1.0000C:\Users\MUSTAKIM\AppData\Roaming\Python\Python38\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
Exception ignored in: <function Viewer.__del__ at 0x000002924A495DC0>
Traceback (most recent call last):
  File "C:\Users\MUSTAKIM\AppData\Roaming\Python\Python38\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\MUSTAKIM\AppData\Roaming\Python\Python38\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\MUSTAKIM\AppData\Roaming\Python\Python38\site-packages\pyglet\window\win32\__init__.py", line 319, in c

In [25]:
scores =dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [26]:
#_ = dqn.test(env, nb_episodes=5, visualize=True) # in case we wanna test outr model and visulize it

# Step 4 : Reloading Agent from memory

In [27]:
# But first we need to save our train weight
dqn.save_weights('dqn.weights.h5f', overwrite=True)

In [28]:
#to test, lets delete all mdels and enviroment
del model
del dqn 
del env

In [29]:
#reconstruct again
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model =build_models(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [30]:
# reload our save weight into our model and test again
dqn.load_weights('dqn.weights.h5f')

In [31]:
#now test our enviroment again
_ = dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
